### Define the configuration

In [1]:
# Import the needed libraries
import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
from PIL import Image
from YOLOv3 import utils

# config
SIZE = [416, 416] #input image dimension
# video_path = 0 # if you use camera as input
video_path = "./dataset/demo_video/road2.mp4" # path for video input
classes = utils.read_coco_names('./YOLOv3/coco.names')
num_classes = len(classes)
GIVEN_ORIGINAL_YOLOv3_MODEL = "./YOLOv3/yolov3_gpu_nms.pb" # to use given original YOLOv3
TENSORRT_YOLOv3_MODEL = "./YOLOv3/TensorRT_YOLOv3.pb" # to use the TensorRT optimized model

In [2]:
#vid = cv2.VideoCapture(video_path)
#return_value, frame = vid.read()
#cv2.imshow("cek", frame)

#cv2.waitKey(0)

### Perform inference

In [ ]:
# get input-output tensor
input_tensor, output_tensors = utils.read_pb_return_tensors(tf.get_default_graph(),
                                                            TENSORRT_YOLOv3_MODEL,
                                                            ["Placeholder:0", "concat_9:0", "mul_9:0"])

# perform inference
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as sess:
    vid = cv2.VideoCapture(video_path) # must use opencv >= 3.3.1 (install it by 'pip install opencv-python')
    while True:
        return_value, frame = vid.read()
        #print("return:", return_value, "; shape:", frame)
        #return_value = True
        #frame = cv2.imread('./pictures/dog.jpg', 1)
        if return_value == False:
            print('ret:', return_value)
            #vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
            vid = cv2.VideoCapture(video_path)
            return_value, frame = vid.read()
        if return_value:
            image = Image.fromarray(frame)
        else:
            raise ValueError("No image!")
            
        img_resized = np.array(image.resize(size=tuple(SIZE)), dtype=np.float32)
        img_resized = img_resized / 255.
        prev_time = time.time()

        boxes, scores = sess.run(output_tensors, feed_dict={input_tensor: np.expand_dims(img_resized, axis=0)})
        boxes, scores, labels = utils.cpu_nms(boxes, scores, num_classes, score_thresh=0.4, iou_thresh=0.5)
        image = utils.draw_boxes(image, boxes, scores, labels, classes, SIZE, show=False)

        curr_time = time.time()
        exec_time = curr_time - prev_time
        result = np.asarray(image)
        info = "time:" + str(round(1000*exec_time, 2)) + " ms, FPS: " + str(round((1000/(1000*exec_time)),1))
        cv2.putText(result, text=info, org=(50, 70), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(255, 0, 0), thickness=2)
        #cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("result", result)
        if cv2.waitKey(10) & 0xFF == ord('q'): break



Instructions for updating:
Use tf.gfile.GFile.
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False
ret: False


### Optimize given YOLOv3 model using TensorRT

In [ ]:
# function to read a ".pb" model 
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.30))) as sess:
    frozen_graph = read_pb_graph("./YOLOv3/yolov3_gpu_nms.pb")
    
    # convert (optimize) frozen model to TensorRT model
    your_outputs = ["Placeholder:0", "concat_9:0", "mul_9:0"]
    trt_graph = trt.create_inference_graph(
        input_graph_def=frozen_graph,# frozen model
        outputs=your_outputs,
        max_batch_size=1,# specify your max batch size
        max_workspace_size_bytes=3*(10**9),# specify the max workspace
        precision_mode="FP16") # precision, can be "FP32" (32 floating point precision) or "FP16"
    print("The model is already converted to TensorRT graph")

    #write the TensorRT model to be used later for inference
    with gfile.FastGFile("./YOLOv3/TensorRT_YOLOv3.pb", 'wb') as f:
        f.write(trt_graph.SerializeToString())
    print("TensorRT YOLOv3 model is successfully stored!")